In [1]:
import datetime
import os
import numpy as np
import scipy.io as sio
import torch
from torch import nn, optim
from tqdm import tqdm
from tqdm import trange
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda:0")

In [3]:
def read_data(base_path='./Data_Predict'):  
    """
    Reads all of the .mat files in the given base_path, and returns a dict with the data found there.
    :param BASE_PATH: the path where the INPUT DATA will be read.
    :return: a dict, containing the EES and difference tensors.
    """
    if not os.path.exists(base_path):
        os.makedirs(base_path)
    
    i = 0
    for file in os.listdir(base_path):
        i = i + 1
    pbar = tqdm(total=i)

    data_dict = {}
    for file in os.listdir(base_path):
        num, data_type = file.split('_')
        data_type = data_type.split('.')[0]
        num = int(num)
        if "EES" in data_type:
            tensor_in = sio.loadmat(os.path.join(base_path, file))['EES_value']
            tensor_in = torch.FloatTensor(tensor_in).squeeze(0)
        else:
            tensor_in = sio.loadmat(os.path.join(base_path, file))['Kulonbseg']
            tensor_in = torch.FloatTensor(tensor_in)
        try:
            data_dict[num][data_type] = tensor_in
        except KeyError:
            data_dict[num] = {data_type: tensor_in}
        pbar.update()
    pbar.close()
    
    new_data = []
    for key in data_dict.keys():
        new_data.append(data_dict[key])
    new_datadict = {'test': new_data}
    
    print("Adatbetöltés kész")
    return new_datadict

In [4]:
# Load the data for prediction
dataset = read_data()

100%|██████████| 20/20 [00:02<00:00,  9.03it/s]


Adatbetöltés kész


In [5]:
class Classificator:

    #-------------------------
    # Label   Class
    #-------------------------
    # 0:      Not damaged
    # 1:      Slightly damaged
    # 2:      Damaged
    # 3:      Very damaged
    #-------------------------
    
    def getClassName(self, c):
        if c==0:
            n = "Not Damaged"
        elif c==1:
            n = "Slightly damaged"
        elif c==2:
            n = "Damaged"
        elif c==3:
            n = "Very damaged"
        return n;
    
    def getClass(self, x):
        x = int(x)
        
        if x==0:
            return 0      
        if x<=100/3:
            return 1
        elif  x>100/3 and x<200/3:
            return 2
        else:
            return 3
    
    # Compare Prediction and Label classes
    def isCorrect(self, prediction, label):
        return self.getClass(prediction) == self.getClass(label)

In [6]:
class CarBadnessGuesser(nn.Module):
    def __init__(self, lr=0.01):
        super(CarBadnessGuesser, self).__init__()

        self.valid_freq = 10

        self.model = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=3, kernel_size=(10, 5, 5), stride=(10, 5, 5)),
            nn.BatchNorm3d(3),
            nn.Conv3d(in_channels=3, out_channels=2, kernel_size=5),
            nn.BatchNorm3d(2),
            nn.Conv3d(in_channels=2, out_channels=1, kernel_size=3),
            nn.BatchNorm3d(1),
            nn.AdaptiveMaxPool3d((1, 1, 10)),
        ) 
        
        self.linear = nn.Sequential(
            nn.Linear(in_features=10, out_features=5),
            nn.ReLU(),
            nn.Linear(in_features=5, out_features=1),
            nn.ReLU()
        )
        if torch.cuda.is_available():
            self.linear.cuda()
            self.model.cuda()

        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(list(self.model.parameters()) + list(self.linear.parameters()), lr=lr)
        
        self.classificator = Classificator()

    def forward(self, x):
        conv_out = self.model(x.unsqueeze(0).unsqueeze(0))
        return self.linear(conv_out.squeeze(-2).squeeze(-2))

    def predict(self):
        """
        Runs the evaluation of the network.
        :return: average loss for the test
        """
        t_correct = 0
        total = 0
        average_loss = 0
        step = 0
        
        for step, data in enumerate(dataset['test']):

            with torch.no_grad():
                input_data = data['KUL'].cuda()
                prediction = self(input_data)
                loss = self.loss_fn(prediction, data['EES'].cuda())
                
                print('---------------------------------')
                print()
                print('Predicted: ', prediction.item())
                print('Expected:  ', data['EES'].cuda().item())   
                print()
                prediction_label = self.classificator.getClass(prediction.item())
                print('Predicted Class: ', self.classificator.getClassName(prediction_label))
                expected_label = self.classificator.getClass(data['EES'].cuda())
                print('Expected Class : ', self.classificator.getClassName(expected_label))
                print()
                print('loss: ', loss.item())
                print()
                
                #check if its correct
                c = self.classificator.isCorrect(prediction.item(), data['EES'].cuda().item())
                if(c == True):
                    t_correct += 1;
                total += 1;
                
                average_loss += loss.item()

        #calculate the test accuracy
        t_acc = t_correct/total * 100
        average_loss = average_loss / step+1
        print()
        print('---------------------------------')
        print('Prediction Total Accuracy: ', t_acc, ' %')
        print('Total Average Loss:  ', average_loss)
        print('---------------------------------')
        print()      
        return average_loss

    def load_weights(self):
        self.load_state_dict = torch.load('./weights/model.weights')
        print("NN weights loaded sucessfully")


In [7]:
if __name__ == "__main__":
     
    # instanciate the NN
    net = CarBadnessGuesser()
    torch.backends.cudnn.enabled = False
   
    # Load the model and Test
    net.load_weights()
    net.predict()
    

NN weights loaded sucessfully


C:\Users\igorr\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


---------------------------------

Predicted:  0.9453139901161194
Expected:   21.408571243286133

Predicted Class:  Not Damaged
Expected Class :  Slightly damaged

loss:  418.744873046875

---------------------------------

Predicted:  1.2955162525177002
Expected:   9.9896240234375

Predicted Class:  Slightly damaged
Expected Class :  Slightly damaged

loss:  75.58751678466797

---------------------------------

Predicted:  0.9050095677375793
Expected:   14.539848327636719

Predicted Class:  Not Damaged
Expected Class :  Slightly damaged

loss:  185.90882873535156

---------------------------------

Predicted:  1.137472152709961
Expected:   49.456153869628906

Predicted Class:  Slightly damaged
Expected Class :  Damaged

loss:  2334.69482421875

---------------------------------

Predicted:  1.0019783973693848
Expected:   2.1053519248962402

Predicted Class:  Slightly damaged
Expected Class :  Slightly damaged

loss:  1.2174330949783325

---------------------------------

Predicted:  1